In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Parkinsons_Disease/GSE227990'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A pilot study of ex vivo human prefrontal RNA transcriptomics in Parkinson’s disease"
!Series_summary	"Parkinson’s disease (PD) can dramatically change cortical neurophysiology. The molecular basis for PD-related cortical changes is unclear because gene expression data are usually derived from postmortem tissue collected at the end of a complex disease and they profoundly change in the minutes after death. Here, we studied cortical changes in tissue from the prefrontal cortex of living PD patients undergoing deep-brain stimulation implantation surgery. We examined 780 genes using the NanoString nCounter platform and found that 40 genes were differentially expressed between PD (n = 12) and essential tremor (ET; n = 9) patients. One of these 40 genes, STAT1, correlated with intraoperative 4-Hz rhythms and intraoperative performance of an oddball reaction-time task. Using a pre-designed custom panel of 780 targets, we compared these intraoperative da

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Gene expression data is available as per the series summary

# Variable availability
sample_characteristics_dict = {
    0: ['tissue: prefrontal cortex'], 
    1: ['cohort: intraoperative-1', 'cohort: intraoperative-2', 'cohort: postmortem'], 
    2: ['diagnosis: ET', 'diagnosis: PD', 'diagnosis: Control'], 
    3: ['age (yr): 73', 'age (yr): 78', 'age (yr): 69', 'age (yr): 67', 'age (yr): 70', 'age (yr): 77', 'age (yr): 71', 'age (yr): 79', 'age (yr): 61', 'age (yr): 65', 'age (yr): 66', 'age (yr): 62', 'age (yr): 63', 'age (yr): 72', 'age (yr): 74', 'age (yr): 84', 'age (yr): 76', 'age (yr): 86'], 
    4: ['Sex: F', 'Sex: M']
}

if 'diagnosis: ET' in sample_characteristics_dict[2]:
    trait_row = 2
    
if any('age (yr):' in val for val in sample_characteristics_dict[3]):
    age_row = 3 

if 'Sex: F' in sample_characteristics_dict[4]:
    gender_row = 4

# Define the conversion functions
def convert_trait(value):
    val = value.split(':')[1].strip()
    if val == "PD":
        return 1
    elif val == "ET" or val == "Control":
        return 0
    return None
    
def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(':')[1].strip()
    if val == "M":
        return 1
    elif val == "F":
        return 0
    return None

# Save cohort information
save_cohort_info('GSE227990', './preprocessed/Parkinsons_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Parkinsons_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Parkinsons_Disease/trait_data/GSE227990.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM7111949': [0.0, 73.0, 0.0], 'GSM7111950': [1.0, 73.0, 0.0], 'GSM7111951': [0.0, 78.0, 1.0], 'GSM7111952': [1.0, 69.0, 1.0], 'GSM7111953': [0.0, 67.0, 0.0], 'GSM7111954': [1.0, 70.0, 1.0], 'GSM7111955': [0.0, 77.0, 1.0], 'GSM7111956': [1.0, 71.0, 1.0], 'GSM7111957': [0.0, 71.0, 0.0], 'GSM7111958': [0.0, 70.0, 0.0], 'GSM7111959': [1.0, 71.0, 0.0], 'GSM7111960': [1.0, 67.0, 1.0], 'GSM7111961': [0.0, 71.0, 0.0], 'GSM7111962': [1.0, 69.0, 1.0], 'GSM7111963': [0.0, 79.0, 0.0], 'GSM7111964': [0.0, 61.0, 1.0], 'GSM7111965': [1.0, 67.0, 1.0], 'GSM7111966': [1.0, 65.0, 1.0], 'GSM7111967': [1.0, 66.0, 1.0], 'GSM7111968': [1.0, 62.0, 1.0], 'GSM7111969': [1.0, 63.0, 1.0], 'GSM7111970': [0.0, 72.0, 1.0], 'GSM7111971': [0.0, 62.0, 1.0], 'GSM7111972': [1.0, 66.0, 1.0], 'GSM7111973': [1.0, 74.0, 1.0], 'GSM7111974': [0.0, 74.0, 1.0], 'GSM7111975': [1.0, 84.0, 1.0], 'GSM7111976': [0.0, 76.0, 1.0], 'GSM7111977': [0.0, 74.0, 1.0], 'GSM7111978': [1.0, 70.0, 1.0], 'GSM7111979': [1.0, 69.0, 1.0], 'GSM711

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ABAT', 'ABL1', 'ACAA1', 'ACHE', 'ACIN1', 'ACTN1', 'ACVRL1', 'ADAM10',
       'ADCY5', 'ADCY8', 'ADCY9', 'ADCYAP1', 'ADORA1', 'ADORA2A', 'ADRA2A',
       'ADRB2', 'AGER', 'AIF1', 'AKT1', 'AKT1S1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# No code needed for STEP6 as requires_gene_mapping is False


Gene annotation preview:
{'ID': ['NEG_A(0)', 'NEG_B(0)', 'NEG_C(0)', 'NEG_D(0)', 'NEG_E(0)'], 'GB_ACC': [nan, nan, nan, nan, nan], 'SPOT_ID': ['ERCC_00096.1', 'ERCC_00041.1', 'ERCC_00019.1', 'ERCC_00076.1', 'ERCC_00098.1']}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Parkinsons_Disease/gene_data/GSE227990.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Parkinsons_Disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE227990', './preprocessed/Parkinsons_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Parkinsons_Disease/GSE227990.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Parkinsons_Disease', the least common label is '0.0' with 15 occurrences. This represents 45.45% of the dataset.
The distribution of the feature 'Parkinsons_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 67.0
  50% (Median): 70.0
  75%: 74.0
Min: 61.0
Max: 86.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 8 occurrences. This represents 24.24% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

